In [ ]:
import cv2
import numpy as np
# import scipy
# from scipy import fftpack
# from scipy.fftpack import dct, idct,fftn,ifftn
from PIL import Image
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch import optim
import os
import cvxpy as cp

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from scipy.fft import dct
from scipy.fft import idctn
from scipy.fft import idct
from scipy.fft import dctn
from scipy.fft import fftn
from scipy.fft import ifftn
from scipy.sparse import random as sparse_random
from scipy.optimize import minimize
import cvxpy as cp

In [ ]:
def normalize(img):
    min_ = np.min(img)
    max_ = np.max(img)
    goal = (2**8-1) * (img - min_) / (max_ - min_) if (max_ - min_) != 0 else img
    goal = np.round(goal).astype(np.uint8)
    return goal
def bilateral(image,dia,sig_clr,sig_spc):
    return (cv2.bilateralFilter(image,dia,sig_clr,sig_spc))

In [ ]:
def CS(image,ratio,alpha,size_d):
    assert(np.shape(image) == (size_d,size_d))
    #plt.imshow(image)
    x = image.flatten()
    x = np.float32(x)
    s = len(x)
    p = int(s*ratio)
    psi=dct(np.float32(np.eye(s)))
    np.random.seed(73)
    phi = np.random.choice([-1,1],size=(p,s))
    y = phi@x
    alpha_hat = cp.Variable(s)
    objective = cp.Minimize(cp.sum_squares(phi@psi@(alpha_hat).T - y) + alpha*cp.norm(alpha_hat, 1))
    problem = cp.Problem(objective)
    problem.solve()
    #x_hat = idct(alpha_hat.value)
    x_hat = psi@alpha_hat.value
    img_hat = x_hat.reshape(size_d,size_d)
#    plt.imshow(img_hat)
    #img_hat = normalize(img_hat)
    #img_hat = np.round(img_hat).astype(np.uint16)
    return img_hat

In [ ]:
def CS_whole(img,a,sr,d):
    _res_ = np.ones_like(img)
    _res_ = np.float32(_res_)
    x_max,y_max = np.shape(img)
    #print(x_max,y_max)
    x = 0
    y = 0
    while (x<x_max):
        y = 0
        while(y<y_max):
            _res_[x:x+d,y:y+d] = CS(image=img[x:x+d,y:y+d],ratio=sr,alpha=a,size_d=d)#调用CS分块处理
            y += d
            #print(x,y)
        x += d
    return _res_

In [ ]:
def CS_simulation(image_name,sr):
    image_path=os.path.join(root_dir, 'constructed_images',image_name)
    #image = np.array(Image.open(image_path).convert("L"))
    image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    image = np.float32(image)
    res64 = CS_whole(img=image,a=10,sr=0.2,d=64)
    
    return res64

In [ ]:
root_dir="face_dataset/train"
raw_image_dir = os.path.join(root_dir, 'raw_images')
constructed_image_dir = os.path.join(root_dir, 'constructed_images')
all_files = os.listdir(constructed_image_dir)
image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
image_files.sort()
#openimg()
for idx, name in enumerate(image_files):
    CS_simulation(name,0.2).save(os.path.join(raw_image_dir, name))
